# SQL Murder Mystery

## Can you find out whodunnit?

![A decorative illustration of a detective looking at an evidence board.](https://mystery.knightlab.com/174092-clue-illustration.png)

There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

## SQL sleuths start here

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **​murder​**that occurred sometime on ​**Jan.15, 2018​** and that it took place in ​**SQL City​**. Start by retrieving the corresponding crime scene report from the police department’s database.

### Exploring the Database Structure

Experienced SQL users can often use database queries to infer the structure of a database. But each database system has different ways of managing this information. The SQL Murder Mystery is built using SQLite. Use this SQL command to find the tables in the Murder Mystery database.

Run this query to find the names of the tables in this database.

SQLite-specific tip: the `sqlite_master` table is SQLite's catalog of tables and schemas, and other databases expose metadata differently.

This command is specific to SQLite. For other databases, you'll have to learn their specific syntax.


In [5]:
# Install required packages
%pip install jupysql sqlalchemy pandas --quiet

# Load SQL magic
%load_ext sql

# Connect to the database
%sql sqlite:///sql-murder-mystery.db
#%config SqlMagic.style = 'table'

Note: you may need to restart the kernel to use updated packages.
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
%%sql
SELECT name
FROM sqlite_master

Running query in 'sqlite:///sql-murder-mystery.db'

name
crime_scene_report
drivers_license
facebook_event_checkin
interview
get_fit_now_member
sqlite_autoindex_get_fit_now_member_1
get_fit_now_check_in
solution
check_solution
income



Besides knowing the table names, you need to know how each table is structured. The way this works is also dependent upon which database technology you use. Here's how you do it with SQLite.

Run this query to find the structure of the `crime_scene_report` table

Change the value of 'name' to see the structure of the other tables you learned about with the previous query.


In [3]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'crime_scene_report'

Running query in 'sqlite:///sql-murder-mystery.db'

sql
"CREATE TABLE crime_scene_report ( date integer, type text, description text, city text )"



### The rest is up to you!

If you're really comfortable with SQL, you can probably get it from here. To help, here is the schema diagram:

![schema diagram](schema.png)

Use your knowledge of the database schema and SQL commands to find out who committed the murder.
### Check your solution

Did you find the killer? When you think you know the answer, submit your suspect using the following code and find out if you're right.


In [18]:
%%sql
INSERT INTO solution VALUES (1, 'Jeremy Bowers');
SELECT value FROM solution;

Running query in 'sqlite:///sql-murder-mystery.db'

1 rows affected.

value
"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."


In [7]:
%%sql
-- Step 1: Find the crime scene report for the murder on Jan 15, 2018 in SQL City
SELECT *
FROM crime_scene_report
WHERE date = 20180115 AND city = 'SQL City' AND type = 'murder'

Running query in 'sqlite:///sql-murder-mystery.db'

date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [8]:
%%sql
-- Step 2: Find the witnesses
-- First witness: last house on Northwestern Dr
SELECT *
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [9]:
%%sql
-- Second witness: Annabel on Franklin Ave
SELECT *
FROM person
WHERE name LIKE 'Annabel%' AND address_street_name = 'Franklin Ave'

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn
16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [10]:
%%sql
-- Step 3: Get the witness interviews
SELECT i.person_id, p.name, i.transcript
FROM interview i
JOIN person p ON i.person_id = p.id
WHERE p.name IN ('Morty Schapiro', 'Annabel Miller')

Running query in 'sqlite:///sql-murder-mystery.db'

person_id,name,transcript
14887,Morty Schapiro,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
16371,Annabel Miller,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [13]:
%%sql
-- Find suspects - gold members with membership starting with 48Z
SELECT p.id, p.name, gm.id, gm.membership_status
FROM get_fit_now_member gm
JOIN person p ON gm.person_id = p.id
WHERE gm.id LIKE '48Z%' AND gm.membership_status = 'gold'

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,id_1,membership_status
28819,Joe Germuska,48Z7A,gold
67318,Jeremy Bowers,48Z55,gold


In [14]:
%%sql
-- Check who was at the gym on January 9th (20180109)
SELECT gm.person_id, p.name, gcheck.check_in_date
FROM get_fit_now_check_in gcheck
JOIN get_fit_now_member gm ON gcheck.membership_id = gm.id
JOIN person p ON gm.person_id = p.id
WHERE p.name IN ('Joe Germuska', 'Jeremy Bowers') 
  AND gcheck.check_in_date = 20180109

Running query in 'sqlite:///sql-murder-mystery.db'

person_id,name,check_in_date
28819,Joe Germuska,20180109
67318,Jeremy Bowers,20180109


In [17]:
%%sql
-- Check car plates for both suspects
SELECT p.name, dl.plate_number
FROM drivers_license dl
JOIN person p ON dl.id = p.license_id
WHERE p.name IN ('Joe Germuska', 'Jeremy Bowers')

Running query in 'sqlite:///sql-murder-mystery.db'

name,plate_number
Jeremy Bowers,0H42W2
